In [1]:
import my_utils.loadModel as lm
import my_utils.initialize_pruning as ip
import torch

In [2]:
#t = Tensor to be prune, n is ln normalization, dim dimension over which we want to perform 
def compute_distance_score(t, n=1, dim_to_keep=[0,1],threshold=1):
        # dims = all axes, except for the one identified by `dim`        
        dim_to_prune = list(range(t.dim()))   #initially it has all dims
        #remove dim which we want to keep from dimstoprune
        for i in range(len(dim_to_keep)):   
            dim_to_prune.remove(dim_to_keep[i])
        
        size = t.shape
        print(f"\nShape of the tensor: {size}")
        print(f"Print the Dims we want to keep: {dim_to_keep}")
        
        module_buffer = torch.zeros_like(t)
                
        #shape of norm should be equal to multiplication of dim to keep values
        norm = torch.norm(t, p=n, dim=dim_to_prune)
        print(f"norm shape = {norm.shape}")
        size = t.shape
        print("Number Of Features Map in current  layer l     =",size[0])
        print("Number Of Features Map in previous layer (l-1) =",size[1])
        
        for i in range(size[0]):
            for j in range(size[1]):
                module_buffer[i][j] = t[i][j]/norm[i][j]
        
        dist = torch.zeros(size[1],size[0],size[0])
        
        channelList = []
        for j in range(size[1]):
            idxtupple = []
            print('.',end='')
            for i1 in range(size[0]):
                for i2 in range((i1+1),size[0]):
                    dist[j][i1][i2] = torch.norm( (module_buffer[i1][j]-module_buffer[i2][j]) ,p=1)
                    dist[j][i2][i1] = dist[j][i1][i2]
                    
                    if dist[j][i1][i2] < threshold:
                        idxtupple.append([j,i1,i2,dist[j][i1][i2]])
            channelList.append(idxtupple)
        return channelList

In [3]:
def sort_kernel_by_distance(kernalList):
    for i in range(len(kernalList)):
        iListLen = len(kernalList[i])
        #print(f'lemgth of list {i} ={iListLen}')
        for j in range(iListLen):
            for k in range(iListLen-j-1):
                #print(f"Value of i={i}     Value of j={j} Value of k={k}")
                if kernalList[i][k+1][3] < kernalList[i][k][3]:
                    kernalList[i][k+1], kernalList[i][k] = kernalList[i][k], kernalList[i][k+1]

In [1]:
def get_k_element(channel_list,k):
    channel_k_list = []
    for i in range(len(channel_list)):
        tempList = []
        for j in range(k):
            tempList.append(channel_list[i][j])
        channel_k_list.append(tempList)
    return channel_k_list

In [5]:
#t = Tensor to be prune, n is ln normalization, dim dimension over which we want to perform 
def compute_kernal_score(t, n=1, dim_to_keep=[0,1],threshold=1):
        # dims = all axes, except for the one identified by `dim`        
        dim_to_prune = list(range(t.dim()))   #initially it has all dims
        
        #remove dim which we want to keep from dimstoprune
        for i in range(len(dim_to_keep)):   
            dim_to_prune.remove(dim_to_keep[i])
        
        size = t.shape
        print(size)
        print(dim_to_keep)
        
        module_buffer = torch.zeros_like(t)
        #sshape of norm should be equal to multiplication of dim to keep values
        norm = torch.norm(t, p=n, dim=dim_to_prune)
        kernelList = []
        size = norm.shape
        for i in range(size[0]):
            for j in range(size[1]):
                kernelList.append([i,j,norm[i][j]])
            
        return kernelList

In [6]:
def sort_kernel_by_value(kernelList):
    return kernelList

In [7]:
def displayLayer(channelTupple):
    for i in range(len(channelTupple)):
        for j in range(len(channelTupple[i])):
            if j%3==0:
                print()
            print(channelTupple[i][j],'\t',end='')  

In [8]:
model_name = 'vgg16'
model = lm.load_model(model_name=model_name,number_of_class=6,pretrainval=True)

VGG16 Loaded
Linear(in_features=4096, out_features=1000, bias=True)


In [9]:
blocks = ip.getBlockList(modelname=model_name)
feature_list = ip.getFeatureList(modelname=model_name)
module = ip.getPruneModule(model,prunelist=feature_list)
prune_count = ip.getPruneCount(module=module,blocks=blocks,maxpr=0.25)
print(f"blocks            = {blocks} \n"
      f"feature list      = {feature_list} \n"
      f"prune count list  = {prune_count}")


blocks            = [2, 2, 3, 3, 3] 
feature list      = [0, 2, 5, 7, 10, 12, 14, 17, 19, 21, 24, 26, 28] 
prune count list  = [3, 3, 8, 8, 21, 21, 21, 64, 64, 64, 128, 128, 128]


In [10]:
channelTuppleList = []
st =2
en = 4
for i in range(st,en):
    channelTuppleList.append(_compute_distance_score(module[i]._parameters['weight'],threshold=1))
print("\n\n\nHere is the :",len(channelTuppleList))


Shape of the tensor: torch.Size([128, 64, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([128, 64])
Number Of Features Map in current  layer l     = 128
Number Of Features Map in previous layer (l-1) = 64
................................................................
Shape of the tensor: torch.Size([128, 128, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([128, 128])
Number Of Features Map in current  layer l     = 128
Number Of Features Map in previous layer (l-1) = 128
................................................................................................................................


Here is the : 2


In [14]:
for i in range(len(channelTuppleList)):
    for j in range(len(channelTuppleList[i])):
        print(f"\n\nlength of list: {len(channelTuppleList[i][j])} and 1st 3 ele are\n{(channelTuppleList[i][j][0:3])}")


length of list: 1434 and 1st 3 ele are
[[0, 0, 1, tensor(0.6386, grad_fn=<AsStridedBackward0>)], [0, 0, 9, tensor(0.4942, grad_fn=<AsStridedBackward0>)], [0, 0, 10, tensor(0.9175, grad_fn=<AsStridedBackward0>)], [0, 0, 12, tensor(0.9868, grad_fn=<AsStridedBackward0>)]]
length of list: 1326 and 1st 3 ele are
[[1, 0, 1, tensor(0.7562, grad_fn=<AsStridedBackward0>)], [1, 0, 3, tensor(0.9557, grad_fn=<AsStridedBackward0>)], [1, 0, 7, tensor(0.6322, grad_fn=<AsStridedBackward0>)], [1, 0, 13, tensor(0.8344, grad_fn=<AsStridedBackward0>)]]
length of list: 1229 and 1st 3 ele are
[[2, 0, 2, tensor(0.7783, grad_fn=<AsStridedBackward0>)], [2, 0, 15, tensor(0.5433, grad_fn=<AsStridedBackward0>)], [2, 0, 17, tensor(0.9211, grad_fn=<AsStridedBackward0>)], [2, 0, 24, tensor(0.9468, grad_fn=<AsStridedBackward0>)]]
length of list: 1240 and 1st 3 ele are
[[3, 0, 24, tensor(0.6401, grad_fn=<AsStridedBackward0>)], [3, 0, 27, tensor(0.9331, grad_fn=<AsStridedBackward0>)], [3, 0, 31, tensor(0.9475, grad_fn=

In [15]:
for i in range(len(channelTuppleList)):
    sortKernalByDistance(channelTuppleList[i])

In [17]:
for i in range(len(channelTuppleList)):
    for j in range(len(channelTuppleList[i])):
        print(f"\n\nlength of list: {len(channelTuppleList[i][j])} and 1st 3 ele are\n{(channelTuppleList[i][j][0:3])}")




length of list: 1434 and 1st 3 ele are
[[0, 73, 121, tensor(0.1909, grad_fn=<AsStridedBackward0>)], [0, 73, 91, tensor(0.2048, grad_fn=<AsStridedBackward0>)], [0, 65, 121, tensor(0.2312, grad_fn=<AsStridedBackward0>)]]


length of list: 1326 and 1st 3 ele are
[[1, 60, 109, tensor(0.2944, grad_fn=<AsStridedBackward0>)], [1, 94, 121, tensor(0.3394, grad_fn=<AsStridedBackward0>)], [1, 0, 47, tensor(0.3715, grad_fn=<AsStridedBackward0>)]]


length of list: 1229 and 1st 3 ele are
[[2, 80, 123, tensor(0.2329, grad_fn=<AsStridedBackward0>)], [2, 3, 123, tensor(0.2407, grad_fn=<AsStridedBackward0>)], [2, 73, 123, tensor(0.2851, grad_fn=<AsStridedBackward0>)]]


length of list: 1240 and 1st 3 ele are
[[3, 109, 124, tensor(0.2520, grad_fn=<AsStridedBackward0>)], [3, 65, 97, tensor(0.2712, grad_fn=<AsStridedBackward0>)], [3, 38, 39, tensor(0.2882, grad_fn=<AsStridedBackward0>)]]


length of list: 856 and 1st 3 ele are
[[4, 15, 47, tensor(0.3427, grad_fn=<AsStridedBackward0>)], [4, 65, 68, tenso

In [ ]:
for i in range(len(channelTuppleList)):
    print("\n\nRow :",i)
    for j in range(3):
        for k in range(len(channelTuppleList[i][j])):
            print(channelTuppleList[i][j][k])

In [ ]:
newList = []
for i in range(len(channelTuppleList)):
    newList.append(get_k_element(channel_list=channelTuppleList[i],k=prune_count[i]) )

for i in range(len(newList)):
    print(f"\n\n\nlenth of list: {len(newList[i])}")
    for j in range(len(newList[i])):
        
        for k in range(len(newList[i][j])):
            print(newList[i][j][k])

In [ ]:
for i in range(len(channelTuppleList)):
    print("\n**************************************************************************************************************************")
    displayLayer(channelTuppleList[i])